In [1]:
import pandas as pd#
import requests
from bs4 import BeautifulSoup
import re
import numpy as np
import lightgbm as lgb
from matplotlib import pyplot as plt
import seaborn as sns
from ydata_profiling import ProfileReport

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder
from sklearn.linear_model import Lasso, Ridge, LinearRegression, ElasticNet
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestRegressor

import xgboost as xgb
import mlflow
import mlflow.sklearn
import mlflow.xgboost

# Preprocessing & Feature Engineering


In [ ]:
def preprocess_data (df): 
    df.replace('""', np.nan, inplace=True)
    df.dropna(inplace=True)
    df["LivingSpace"] = pd.to_numeric(df["LivingSpace"], errors="coerce")
    df["Rooms"] = pd.to_numeric(df["Rooms"], errors="coerce")
    df.dropna(subset=['ConstructionYear'], inplace=True)
    df.dropna(subset=['Object_price'], inplace=True)
    df.dropna(subset=['Rooms'], inplace=True)
    df.dropna(subset=['LivingSpace'], inplace=True)
    df.drop('Unnamed: 0', axis=1, inplace=True)
    df['ZipCode']= df['ZipCode'].astype(str)
    df['LivingSpace']= df['LivingSpace'].astype(float)
    df = df.reindex()
    df = df.reset_index(drop=True)

    # keep only the most promising features
    # sns.heatmap(df.corr(), cmap="RdBu")
    important_num_cols = list(df.corr()["Object_price"][(df.corr()["Object_price"]>0.20) | (df.corr()["Object_price"]<-0.20)].index)
    cat_cols = [col for col in df.columns if df[col].dtype == 'object']
    important_cols = important_num_cols + cat_cols
    df = df[important_cols]

    # one-hot encoding
    y = df['Object_price']
    X = df.drop('Object_price', axis=1)
    X = pd.get_dummies(X, columns=cat_cols)

    return X, y

# Explorative Data Analysis

In [ ]:
df = pd.read_excel(r'data\flats_to_rent_wue_preprocessed_0407.xlsx')
profile = ProfileReport(df, title="Flats -  Würzburg - Rent - Overview", explorative=True)
profile.to_file("eda-wue-rent-all.html")

# Data Split


In [ ]:
def data_split(X, y, train_size=0.8, random_state=42):
    train_X, test_X, train_y, test_y = train_test_split(X, y, 
                                                  train_size=train_size, 
                                                  random_state = random_state)
    train_X, val_X, train_y, val_y = train_test_split(train_X, train_y, 
                                                  train_size=train_size, 
                                                  random_state = random_state)
    return train_X, test_X, train_y, test_y, val_X, val_y

# Training, Evaluation and Logging Of Models

In [ ]:
mlflow.set_tracking_uri("http://localhost:5000")

## Baseline 

In [ ]:
def scrape_avg_rental_prices():
    url = "https://www.wohnungsboerse.net/mietspiegel-Wuerzburg/2772"
    response = requests.get(url)
    response.raise_for_status()
    soup = BeautifulSoup(response.text, "html.parser")
    script_tag = soup.find("script", string=lambda text: "pdfData" in text)
    rental_price = 0
    if script_tag:
        script_content = script_tag.string
        start_index = script_content.find("avg_rent_price: ") + len("avg_rent_price: '")
        end_index = script_content.find("',", start_index)
        rental_price = script_content[start_index:end_index]
        rental_price = (
            rental_price.replace("€/m2", "").replace(".", "").replace(",", ".")
        )
        rental_price = rental_price.strip()
        rental_price = float(rental_price)
        print(f"Extrcated rental price per square meter via scraper: {rental_price}")
    else:
        print("The script tag containing the rental price was not found.")
    return rental_price

In [ ]:
def scrape_avg_buy_prices():
    url = "https://www.wohnungsboerse.net/immobilienpreise-Wuerzburg/2772"
    response = requests.get(url)
    response.raise_for_status()
    soup = BeautifulSoup(response.text, "html.parser")
    p_element = soup.find("p", class_="mb-8")
    buy_price = 0
    if p_element:
        pattern = r"\d{1,3}(?:\.\d{3})*(?:,\d{2})?€/m²"
        match = re.search(pattern, p_element.text)
        if match:
            buy_price = match.group()
            buy_price = buy_price.replace("€/m²", "").replace(".", "").replace(",", ".")
            print(f"Extrcated buy price per square meter via scraper: {buy_price}")
        else:
            print("Price not found")
    else:
        print("The element ontaining the buy price was not found.")
    return buy_price

In [ ]:
def baseline_avg_rent_price(val_X, val_y, runname="baseline_rent"):
    avg_price_per_sqm_rent = scrape_avg_rental_prices()
    print(f"Average rental price per sqm: {avg_price_per_sqm_rent}")
    baseline_preds = val_X['LivingSpace'] * avg_price_per_sqm_rent
    baseline_mae = mean_absolute_error(val_y, baseline_preds)
    baseline_r2 = r2_score(val_y, baseline_preds)
    baseline_mse = mean_squared_error(val_y, baseline_preds)

    with mlflow.start_run(run_name=runname):
        mlflow.log_metric("mse", baseline_mse)
        mlflow.log_metric("mae", baseline_mae)
        mlflow.log_metric("r2", baseline_r2)

    print(f"Baseline Mae: {baseline_mae}")
    print(f"Baseline MSE: {baseline_mse}")    
    print(f"Baseline R2 Score: {baseline_r2}")
    return avg_price_per_sqm_rent, baseline_mae, baseline_mse, baseline_r2

In [ ]:
def baseline_avg_buy_price(val_X, val_y, runname="baseline_buy"):
    avg_price_per_sqm_buy = scrape_avg_buy_prices()
    print(f"Average rental price per sqm: {avg_price_per_sqm_buy}")
    
    baseline_preds = val_X['LivingSpace'] * avg_price_per_sqm_buy
    baseline_mae = mean_absolute_error(val_y, baseline_preds)
    baseline_r2 = r2_score(val_y, baseline_preds)
    baseline_mse = mean_squared_error(val_y, baseline_preds)

    with mlflow.start_run(run_name=runname):
        mlflow.log_metric("mse", baseline_mse)
        mlflow.log_metric("mae", baseline_mae)
        mlflow.log_metric("r2", baseline_r2)

    print(f"Baseline Mae: {baseline_mae}")
    print(f"Baseline MSE: {baseline_mse}")    
    print(f"Baseline R2 Score: {baseline_r2}")

    return avg_price_per_sqm_buy, baseline_mae, baseline_mse, baseline_r2

## Linear Regression

In [ ]:
def train_and_eval_linear(X_train, y_train, X_val, y_val, runname ="linear-regression"):
    mlflow.sklearn.autolog()
    with mlflow.start_run(run_name=runname):
        model = LinearRegression()
        model.fit(X_train, y_train)
        
        pred_train = model.predict(X_train)
        mae_train = mean_absolute_error(y_train, pred_train)
        mse_train = mean_squared_error(y_train, pred_train)
        r2_train = r2_score(y_train, pred_train)
        mlflow.log_metric("mae_train", mae_train)
        mlflow.log_metric("mse_train", mse_train)
        mlflow.log_metric("r2_train", r2_train)
        
        y_pred = model.predict(X_val)
        mae = mean_absolute_error(y_val, y_pred)
        mse = mean_squared_error(y_val, y_pred)
        r2 = r2_score(y_val, y_pred)
        mlflow.log_metric("mae", mae_train)
        mlflow.log_metric("mse", mse_train)
        mlflow.log_metric("r2", r2_train)
    return model, mae, mse, r2, mae_train, mse_train, r2_train

## Lasso Regression

In [ ]:
def train_and_eval_lasso(X_train, y_train, X_val, y_val, runname ="lasso-regression"):
    mlflow.sklearn.autolog()
    with mlflow.start_run(run_name=runname):
        model = Lasso()
        model.fit(X_train, y_train)
        
        pred_train = model.predict(X_train)
        mae_train = mean_absolute_error(y_train, pred_train)
        mse_train = mean_squared_error(y_train, pred_train)
        r2_train = r2_score(y_train, pred_train)
        mlflow.log_metric("mae_train", mae_train)
        mlflow.log_metric("mse_train", mse_train)
        mlflow.log_metric("r2_train", r2_train)
        
        y_pred = model.predict(X_val)
        mae = mean_absolute_error(y_val, y_pred)
        mse = mean_squared_error(y_val, y_pred)
        r2 = r2_score(y_val, y_pred)
        mlflow.log_metric("mae", mae_train)
        mlflow.log_metric("mse", mse_train)
        mlflow.log_metric("r2", r2_train)
    return model, mae, mse, r2, mae_train, mse_train, r2_train

## Ridge Regression

In [ ]:
def train_and_eval_ridge(X_train, y_train, X_val, y_val, runname ="ridge-regression"):
    mlflow.sklearn.autolog()
    with mlflow.start_run(run_name=runname):
        model = Ridge()
        model.fit(X_train, y_train)
        
        pred_train = model.predict(X_train)
        mae_train = mean_absolute_error(y_train, pred_train)
        mse_train = mean_squared_error(y_train, pred_train)
        r2_train = r2_score(y_train, pred_train)
        mlflow.log_metric("mae_train", mae_train)
        mlflow.log_metric("mse_train", mse_train)
        mlflow.log_metric("r2_train", r2_train)
        
        y_pred = model.predict(X_val)
        mae = mean_absolute_error(y_val, y_pred)
        mse = mean_squared_error(y_val, y_pred)
        r2 = r2_score(y_val, y_pred)
        mlflow.log_metric("mae", mae_train)
        mlflow.log_metric("mse", mse_train)
        mlflow.log_metric("r2", r2_train)
    return model, mae, mse, r2, mae_train, mse_train, r2_train

## Random Forest

In [ ]:
def train_and_eval_rf(train_X, train_y, val_X, val_y, n_estimators=50, random_state=0, run_name="random-forest"):
    mlflow.sklearn.autolog()
    with mlflow.start_run(run_name=run_name):
        model = RandomForestRegressor(n_estimators=n_estimators, random_state=random_state)
        model.fit(train_X, train_y)
        pred_train = model.predict(train_X)
        mae_train = mean_absolute_error(train_y, pred_train)
        mse_train = mean_squared_error(train_y, pred_train)
        r2_train = r2_score(train_y, pred_train)
        preds= model.predict(val_X)
        mae = mean_absolute_error(val_y, preds)
        mse = mean_squared_error(val_y, preds)
        r2 = r2_score(val_y, preds)
        mlflow.log_metric("mae", mae)
        mlflow.log_metric("mse", mse)
        mlflow.log_metric("r2", r2)
        mlflow.log_metric("mae_train", mae_train)
        mlflow.log_metric("mse_train", mse_train)
        mlflow.log_metric("r2_train", r2_train)
        return model, mae, mse, r2, mae_train, mse_train, r2_train

## XGBRegressor


In [ ]:
def train_and_eval_xgb(train_X, train_y, val_X, val_y, run_name="xgb", early_stopping_rounds=30, max_depth=6, n_estimators=1000):
    mlflow.xgboost.autolog()
    with mlflow.start_run(run_name=run_name):
        model = xgb.XGBRegressor(eval_metric=['rmse', 'mae'], early_stopping_rounds=early_stopping_rounds, random_state=42, max_depth=max_depth, n_estimators=n_estimators)
        mlflow.log_param("early_stopping_rounds", early_stopping_rounds)
        mlflow.log_param("max_depth", max_depth)
        mlflow.log_param("n_estimators", n_estimators)
        model.fit(X=train_X, 
                  y=train_y,
                  eval_set=[(val_X, val_y)],
                  verbose=True
        )

        pred_train = model.predict(train_X)
        mae_train = mean_absolute_error(train_y, pred_train)
        mse_train = mean_squared_error(train_y, pred_train)
        r2_train = r2_score(train_y, pred_train)
        mlflow.log_metric("mae_train", mae_train)
        mlflow.log_metric("mse_train", mse_train)
        mlflow.log_metric("r2_train", r2_train)
        preds= model.predict(val_X)
        mae = mean_absolute_error(val_y, preds)
        mse = mean_squared_error(val_y, preds)
        r2 = r2_score(val_y, preds)
        mlflow.log_metric("mae", mae)
        mlflow.log_metric("mse", mse)
        mlflow.log_metric("r2", r2)        
        return model, mae, mse, r2, mae_train, mse_train, r2_train

## ElasticNet

In [ ]:
def train_and_eval_elasticnet(train_X, train_y, val_X, val_y, runname="elasticNet"):
    mlflow.sklearn.autolog()
    with mlflow.start_run(run_name=runname):
        model = ElasticNet()
        model.fit(train_X, train_y)
        pred_train = model.predict(train_X)
        mae_train = mean_absolute_error(train_y, pred_train)
        mse_train = mean_squared_error(train_y, pred_train)
        r2_train = r2_score(train_y, pred_train)
        preds= model.predict(val_X)
        mae = mean_absolute_error(val_y, preds)
        mse = mean_squared_error(val_y, preds)
        r2 = r2_score(val_y, preds)
        mlflow.log_metric("mae", mae)
        mlflow.log_metric("mse", mse)
        mlflow.log_metric("r2", r2)
        mlflow.log_metric("mae_train", mae_train)
        mlflow.log_metric("mse_train", mse_train)
        mlflow.log_metric("r2_train", r2_train)
        return model, mae, mse, r2, mae_train, mse_train, r2_train

# Complete Pipeline

In [ ]:
def complete_pipeline(df):
    X, y = preprocess_data(df)
    print("Done with preprocessing")
    train_X, val_X, test_X, train_y, val_y, test_y = data_split(X,y)
    print("Done with data split")
    model, mae, mse, r2, mae_train, mse_train, r2_train = train_and_eval_rf(train_X, train_y, val_X, val_y)
    print("Done with rf_train")